In [1]:
# 本代码由可视化策略环境自动生成 2019年4月18日 15:22
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# 回测引擎：每日数据处理函数，每天执行一次
def m19_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]

    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.perf_tracker.position_tracker.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities and not context.has_unfinished_sell_order(equities[x]))])))
        # print('rank order for sell %s' % instruments)
        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)

# 回测引擎：准备数据，只执行一次
def m19_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m19_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5


g = T.Graph({

    'm1': 'M.instruments.v2',
    'm1.start_date': '2010-01-01',
    'm1.end_date': '2015-01-01',
    'm1.market': 'CN_STOCK_A',
    'm1.instrument_list': '',
    'm1.max_count': 0,

    'm2': 'M.advanced_auto_labeler.v2',
    'm2.instruments': T.Graph.OutputPort('m1.data'),
    'm2.label_expr': """# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    'm2.start_date': '',
    'm2.end_date': '',
    'm2.benchmark': '000300.SHA',
    'm2.drop_na_label': True,
    'm2.cast_label_int': True,

    'm3': 'M.input_features.v1',
    'm3.features': """# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
return_5
return_10
return_20
avg_amount_0/avg_amount_5
avg_amount_5/avg_amount_20
rank_avg_amount_0/rank_avg_amount_5
rank_avg_amount_5/rank_avg_amount_10
rank_return_0
rank_return_5
rank_return_10
rank_return_0/rank_return_5
rank_return_5/rank_return_10
pe_ttm_0
""",

    'm15': 'M.general_feature_extractor.v7',
    'm15.instruments': T.Graph.OutputPort('m1.data'),
    'm15.features': T.Graph.OutputPort('m3.data'),
    'm15.start_date': '',
    'm15.end_date': '',
    'm15.before_start_days': 0,

    'm16': 'M.derived_feature_extractor.v3',
    'm16.input_data': T.Graph.OutputPort('m15.data'),
    'm16.features': T.Graph.OutputPort('m3.data'),
    'm16.date_col': 'date',
    'm16.instrument_col': 'instrument',
    'm16.drop_na': False,
    'm16.remove_extra_columns': False,

    'm7': 'M.join.v3',
    'm7.data1': T.Graph.OutputPort('m2.data'),
    'm7.data2': T.Graph.OutputPort('m16.data'),
    'm7.on': 'date,instrument',
    'm7.how': 'inner',
    'm7.sort': False,

    'm13': 'M.dropnan.v1',
    'm13.input_data': T.Graph.OutputPort('m7.data'),

    'm6': 'M.stock_ranker_train.v5',
    'm6.training_ds': T.Graph.OutputPort('m13.data'),
    'm6.features': T.Graph.OutputPort('m3.data'),
    'm6.learning_algorithm': '排序',
    'm6.number_of_leaves': 30,
    'm6.minimum_docs_per_leaf': 1000,
    'm6.number_of_trees': 20,
    'm6.learning_rate': 0.1,
    'm6.max_bins': 1023,
    'm6.feature_fraction': 1,
    'm6.m_lazy_run': False,

    'm9': 'M.instruments.v2',
    'm9.start_date': T.live_run_param('trading_date', '2015-01-01'),
    'm9.end_date': T.live_run_param('trading_date', '2017-01-01'),
    'm9.market': 'CN_STOCK_A',
    'm9.instrument_list': '',
    'm9.max_count': 0,

    'm17': 'M.general_feature_extractor.v7',
    'm17.instruments': T.Graph.OutputPort('m9.data'),
    'm17.features': T.Graph.OutputPort('m3.data'),
    'm17.start_date': '',
    'm17.end_date': '',
    'm17.before_start_days': 0,

    'm18': 'M.derived_feature_extractor.v3',
    'm18.input_data': T.Graph.OutputPort('m17.data'),
    'm18.features': T.Graph.OutputPort('m3.data'),
    'm18.date_col': 'date',
    'm18.instrument_col': 'instrument',
    'm18.drop_na': False,
    'm18.remove_extra_columns': False,

    'm14': 'M.dropnan.v1',
    'm14.input_data': T.Graph.OutputPort('m18.data'),

    'm8': 'M.stock_ranker_predict.v5',
    'm8.model': T.Graph.OutputPort('m6.model'),
    'm8.data': T.Graph.OutputPort('m14.data'),
    'm8.m_lazy_run': False,

    'm19': 'M.trade.v4',
    'm19.instruments': T.Graph.OutputPort('m9.data'),
    'm19.options_data': T.Graph.OutputPort('m8.predictions'),
    'm19.start_date': '',
    'm19.end_date': '',
    'm19.handle_data': m19_handle_data_bigquant_run,
    'm19.prepare': m19_prepare_bigquant_run,
    'm19.initialize': m19_initialize_bigquant_run,
    'm19.volume_limit': 0.025,
    'm19.order_price_field_buy': 'open',
    'm19.order_price_field_sell': 'close',
    'm19.capital_base': 1000000,
    'm19.auto_cancel_non_tradable_orders': True,
    'm19.data_frequency': 'daily',
    'm19.price_type': '后复权',
    'm19.product_type': '股票',
    'm19.plot_charts': True,
    'm19.backtest_only': False,
    'm19.benchmark': '',
})

# g.run({})


def m20_param_grid_builder_bigquant_run():
    param_grid = {}

    # 在这里设置需要调优的参数备选
    param_grid['m6.number_of_trees'] = [5, 10, 20]

    return param_grid

def m20_scoring_bigquant_run(result):
    score = result.get('m19').read_raw_perf()['sharpe'].tail(1)[0]

    return score


m20 = M.hyper_parameter_search.v1(
    param_grid_builder=m20_param_grid_builder_bigquant_run,
    scoring=m20_scoring_bigquant_run,
    search_algorithm='网格搜索',
    search_iterations=10,
    workers=1,
    worker_distributed_run=True,
    worker_silent=True,
    run_now=True,
    bq_graph=g
)


Fitting 1 folds for each of 3 candidates, totalling 3 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] m6.number_of_trees=5 ............................................
[CV] ... m6.number_of_trees=5, score=0.8200535615725041, total=  10.8s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.9s remaining:    0.0s
[CV] m6.number_of_trees=10 ...........................................
[CV] .. m6.number_of_trees=10, score=1.4404307211678549, total=  21.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   32.1s remaining:    0.0s
[CV] m6.number_of_trees=20 ...........................................
[CV] ... m6.number_of_trees=20, score=1.914699392161583, total=  10.4s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   42.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   42.8s finished


In [5]:
m20.result.best_params_

{'m6.number_of_trees': 20}

In [6]:
m20.result.best_score_

1.914699392161583